<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам необходимо защитить данные клиентов страховой компании. Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию и обосновать корректность его работы. 

Требование - необходимо защитить данные таким образом, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

Импортируем необходимые нам для работы библиотеки

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Загрузим данные и быстро взглянем на них

In [2]:
df = pd.read_csv(
    "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1c9pxoZV-S_ajfP5kJmRYIaKD7WW8qm4G"
)

In [3]:
def data_preview(data):
    display(data.head(5))
    print("-" * 100)
    data.info()
    print("-" * 100)
    print("Пропуски в данных")
    display(data.isna().mean().sort_values(ascending=False))
    print("-" * 100)
    print(f"Количество явных дубликатов - {data.duplicated().sum()}")
    print("-" * 100)
    display(data.describe())

In [4]:
data_preview(df)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
----------------------------------------------------------------------------------------------------
Пропуски в данных


Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

----------------------------------------------------------------------------------------------------
Количество явных дубликатов - 153
----------------------------------------------------------------------------------------------------


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [24]:
df.corr()

,sex,age,salary,family_members,insurance_payments
sex,1.000000,0.002074,0.014910,-0.008991,0.010140
age,0.002074,1.000000,-0.019093,-0.006692,0.651030
salary,0.014910,-0.019093,1.000000,-0.030296,-0.014963
family_members,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
insurance_payments,0.010140,0.651030,-0.014963,-0.036290,1.000000


При первичном обзоре данных мы можем отметить следующее:
* нам для преобразования предоставлены данные 5000 клиентов страховой компании
* для каждого клиента указаны признаки: пол, возраст, зарплата, количество членов семьи, а также целевой признак - количество страховых выплат
* пропусков в данных нет
* в данных пристуствуют явные дубликаты, что вполне имеет место быть в данном случае
* в распределении статистик данных на первый взгляд ничего необычного нет
* если говорить о корреляции, то можно отметить сильную прямую связь нашего целевого признака с возрастом клиента, с остальными признаками связей не наблюдается
* единственное, что необходимо подправить для дальнейшей работы - это наименование столбцов (крайне неудобно работать с кириллицей)
* ну и также можно привести все данные в целочисельный формат (изменить тип данных для возраста и зарплаты)

In [5]:
df.columns = ["sex", "age", "salary", "family_members", "insurance_payments"]

Посмотрим не ухудшиться ли точность данных о зарплате при округлении

In [16]:
len(df['salary'][(df['salary'] % 1) != 0]) / len(df)

0.008

In [21]:
(1 - df['salary'] % 1).min()

7.275957614183426e-12

При целочисельном делении мы видим, что не целых чисел в данных о зарплате меньше процента и остаток от деления при этом крайне маленький, значит эти данные можно смело округлить до целых величин

In [25]:
df = df.astype('int')

Также можно уже на данной стадии сразу выделить список признаков и наш таргет

In [26]:
X = df.columns[:-1]
y = df.columns[-1]

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Ответим на вопрос: **"Изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу?"**

Попробуем ответить на этот вопрос с помощью теоретических знаний

Изначально на непреобразованных признаках наши предсказания вычислялись по формуле:
$$
a = Xw
$$

После перемножения признаков (`X`) на матрицу (`A`) формула предсказаний приняла вид:

$$
a` = X A w`
$$

Если качество линейной регрессии при перемножении признаков на матрицу не меняется то справедлива будет формула:

$$
a = a`
$$

$$
Xw = XAw`
$$

Для соблюдения данного равенства необходимо условие:

$$
w` = A^{-1}w
$$

В таком случае при условии что матрица `A` обратимая:

$$
Xw  = XAA^{-1}w = XEw = Xw
$$


Проверим выполняется ли это условие.

Формула обучения для непреобразованных данных:
$$
w = (X^T X)^{-1} X^T y
$$

Формула обучения для преобразованных данных:

$$
w` = ((XA)^T XA)^{-1} (XA)^T y 
$$

Принимая во внимание ассоциативность умножения и свойства обращения и транспонирования произведения матриц
$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$, несколько преобразуем формулу:

$$
w` = ((XA)^T (XA))^{-1} (XA)^T y = (A^TX^T(XA))^{-1} A^TX^T y = (A^T(X^TX)A)^{-1} A^TX^T y = A^{-1}(X^TX)^{-1}(A^T)^{-1} A^T X^T y = A^{-1}(X^TX)^{-1} E X^T y = A^{-1}(X^T X)^{-1} X^T y = A^{-1} w
$$

Зная, что умножение обратной матрицы $(A^T)^{-1}$ на саму матрицу $(A^T)$ равно единичной матрице $E$, а умножение на единичную матрицу равно самой матрице, получаем в конечном итоге что:

$$
w` = A^{-1} w
$$

Как раз условие при котором соблюдается условие равенства предсказаний линейной регрессии:

$$
a = a`
$$

Таким образом можно сделать вывод, что 

**При умножении признаков на произвольную матрицу качество предсказаний линейной регрессии не изменится при условии обратимости этой матрицы**

## Алгоритм преобразования

На предыдущем этапе мы выяснили, что при умножении признаков на произвольную матрицу качество предсказаний линейной регрессии не изменится при условии обратимости этой матрицы.

Соответственно нам для защиты персональных данных клиентов от утечки можно эти данные закодировать, умножив их на произвольную матрицу с условием обратимости этой матрицы.

Напишем функцию кодирования наших данных.

In [27]:
def data_encoding(data, X=X, y=y, rs=1):
    A = data[X].values
    # проверим условие обратимости матрицы
    while True:
        try:
            np.random.seed(rs)
            B = np.random.normal(0, 1, size=(A.shape[1], A.shape[1]))
            np.linalg.inv(B)
            break
        except:
            rs += 1
            continue
    data_encoded = pd.DataFrame(A @ B, columns=X).join(data[y])
    return data_encoded, B

In [28]:
df_encoded, code_matrix = data_encoding(df)
df_encoded.head()

,sex,age,salary,family_members,insurance_payments
0,15861.122805,-12464.129521,72592.696557,-102216.361534,0
1,12162.971984,-9582.329103,55641.496718,-78321.462370,1
2,6724.917838,-5303.522507,30754.866219,-43285.029900,0
3,13321.459031,-10447.845079,61008.809561,-85926.052714,0
4,8352.776166,-6573.621640,38209.343715,-53792.059280,0


Персональные данные клиентов успешно закодированы.

Напишем функцию для декодирования данных

In [29]:
def data_decoding(data, code_matrix, X=X, y=y, rs=1):
    A = data[X].values
    B = code_matrix
    return (
        pd.DataFrame(np.round(A @ np.linalg.inv(B)), columns=X)
        .astype("int")
        .join(data[y])
    )

In [30]:
df_decoded = data_decoding(df_encoded, code_matrix)
df_decoded.head()

,sex,age,salary,family_members,insurance_payments
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


Данные успешно декодированы.

Проверим изменились ли данные по сравнению с первоначальными.

In [31]:
(df == df_decoded).mean()

sex                   1.0
age                   1.0
salary                1.0
family_members        1.0
insurance_payments    1.0
dtype: float64

Данные вернулись в свой первоначальный вид!

Теперь мы можем как закодировать данные, так и вернуть их в первоначальный вид

## Проверка алгоритма

Проверим на практике, что качество линейной регрессии для закодированных таким способом данных не изменится относительно исходных данных.

Для начала разделим наши данные на обучающую и тестовую выборки

In [33]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    X_encoded_train,
    X_encoded_test,
    y_encoded_train,
    y_encoded_test,
) = train_test_split(
    df[X], df[y], df_encoded[X], df_encoded[y], test_size=0.2, random_state=42
)

Обучим модель линейной регрессии на наших исходных данных, сделаем предсказания и оценим качество модели R2 метрикой

In [34]:
model_1 = LinearRegression()
model_1.fit(X_train, y_train)
y_pred = model_1.predict(X_test)
print(
    f"R2 метрика модели линейной регрессии, обученной на исходных данных - {r2_score(y_test, y_pred):.7f}"
)

R2 метрика модели линейной регрессии, обученной на исходных данных - 0.4368695


Повторим эти операции для преобразованных данных

In [35]:
model_2 = LinearRegression()
model_2.fit(X_encoded_train, y_encoded_train)
y_encoded_pred = model_2.predict(X_encoded_test)
print(
    f"R2 метрика модели линейной регрессии, обученной на преобразованных данных - {r2_score(y_encoded_test, y_encoded_pred):.7f}"
)


R2 метрика модели линейной регрессии, обученной на преобразованных данных - 0.4368695


Как мы можем видеть на практике качество модели линейной регрессии не изменилось при кодировки данных, что мы и доказывали теоретически

## Общий вывод

**Целью нашей работы** была защита персональных данных клиентов страховой компании. Необходимо было разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. **Требование** - нужно защитить данные таким образом, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

На начальном этапе мы загрузили данные и первично ознакомились с ними. 

Убедившись, что с данными все в порядке, мы теоретически разобрали метод кодирования данных с помощью умножения данных на матрицу.

Доказав в теории, что при условии обратимости этой матрицы, соблюдается наше главное требование о неизменности качества модели линейной регрессии, мы написали функции для кодирования и декодирования данных (умножая их сначала на произвольную матрицу, а потом на обратную ей соответственно). 

Убедившись что данные успешно кодируются и декодируются, мы обучили модели регрессии, сделали предсказания и оценили R2 метрику модели сначала для исходных данных, а затем и для закодированных данных. И в этот раз мы уже на практике подтвердили наше теоретически доказанное предположение о неизменности качества модели линейной регрессии при умножении признаков на обратимую матрицу.

Исходя из всего вышеописанного, **для защиты персональных данных клиентов страховой компании подойдет метод перемножения данных на произвольную обратимую матрицу. При таком методе соблюдается условие сохранения качества модели машинного обучения.**